In [192]:
import pandas as pd

In [193]:
raw_data = pd.read_csv("../PlayBrush/Raw_data/1_rawdata.csv")
groups = pd.read_csv("../PlayBrush/Raw_data/2_groups.csv")

## General assumptions: (dataset cleaning)

In [194]:
raw_data.head(2)

,PlaybrushID,TimestampUTC,UpTime,DownTime,LeftTime,RightTime,NoneTime,Unnamed: 7
0,PB2500017115,Mon Sep 18 2017 08:03:18 GMT+0100 (BST),6.6,0.1,0.3,0.3,5.7,NaN
1,PB2500017115,Mon Sep 18 2017 08:06:32 GMT+0100 (BST),0.0,3.8,1.8,0.0,1.4,NaN


In [195]:
raw_data = raw_data[~pd.isna(raw_data.PlaybrushID)]

raw_data['TimestampUTC'] = pd.to_datetime(raw_data['TimestampUTC'])


In [197]:
# Morning and evening sessions

raw_data['hour'] = raw_data['TimestampUTC'].dt.hour
time_of_day = lambda row: 'evening' if row.hour > 14 else 'morning'
raw_data['sessions'] = raw_data.apply(time_of_day, axis = 1)



In [200]:
# Merge brush sessions that are less than 2 minutes apart into a single brush session
raw_data = raw_data.sort_values(['PlaybrushID', 'TimestampUTC'])
raw_data['one_session_flag'] = ((raw_data.PlaybrushID == raw_data.PlaybrushID.shift())
                            & ( raw_data.sessions == raw_data.sessions.shift())
                            & ((raw_data.TimestampUTC - raw_data.TimestampUTC.shift()).astype('timedelta64[m]') < 2))

raw_data['session_id'] = (raw_data.one_session_flag == False).cumsum()
        
        

In [201]:
# The total length of a brush session is the sum of the up, down, left, right and none times.

time_brush = raw_data[['UpTime', 'DownTime', 'LeftTime', 'RightTime', 'NoneTime']]

raw_data['sub_total_time'] = time_brush.sum(axis =1)

raw_data = raw_data.merge(raw_data.groupby(['PlaybrushID', 'session_id']).sub_total_time.sum().rename('total_time')
                        , on = ['PlaybrushID', 'session_id'])

In [202]:
#  Discard brush sessions that are less than 20 seconds in total
raw_data = raw_data[raw_data.total_time >= 20]

In [239]:
#  When a user brushes multiple times in a morning or an evening, record the longest brush and discard
#  the others

raw_data['day'] = raw_data.TimestampUTC.dt.date

dataset = (raw_data.merge(raw_data.groupby(['PlaybrushID', 'day', 'sessions']).total_time.max()
                        , on = ['PlaybrushID', 'day', 'sessions', 'total_time'], how = 'inner')
                           [['PlaybrushID','day','sessions', 'total_time']]
                        ).drop_duplicates()


## User Information

In [318]:
# User information

#How many times the user brushed in the morning, and in the evening for each day of the week. 0 for no
#brush, 1 for morning or evening brush, and 2 for morning and evening brush. Multiple brushes in the
#same morning should only be counted once.

dataset['day'] = pd.to_datetime(dataset['day'])
dataset['dow'] = dataset['day'].dt.day_name()

num_of_session_dow = (pd.DataFrame(dataset.groupby(['PlaybrushID','dow'])
                        .sessions.count().rename('num_sessions')).reset_index())

#How many days in the week a user brushed twice a day. Again, twice a day represents morning and
#evening, and not just multiple brushes.

num_twice_day = (num_of_session_dow[num_of_session_dow.num_sessions >1].
                groupby('PlaybrushID').dow.count().rename('total-brushes'))


# The total number of valid morning and evening brush sessions in the week.

dataset['week'] = dataset['day'].dt.week
num_week = dataset.groupby(['PlaybrushID','week']).sessions.count().rename('twice-brushes')


# The average time spent brushing per valid session in the week.
avr_time_session = dataset.groupby(['PlaybrushID','week']).total_time.mean().rename('vg-brush-time')

/var/folders/9t/5gcr47m91839pxy0fm6mhjw00000gn/T/ipykernel_24832/620537038.py:22: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataset['week'] = dataset['day'].dt.week


In [322]:
final_table = (num_of_session_dow.pivot(index='PlaybrushID', 
                        columns='dow', values='num_sessions').reset_index())

final_table = final_table.merge(groups, left_on = 'PlaybrushID', right_on = 'PBID')

final_table = (final_table.merge(num_twice_day, on = 'PlaybrushID')
                        .merge(num_week, on = 'PlaybrushID' )
                        .merge(avr_time_session, on = 'PlaybrushID')
                        .rename(columns = {'Monday': 'mon'
                                          , 'Tuesday' : 'tue'
                                          , 'Wednesday' : 'wed'
                                          , 'Thursday' : 'thu'
                                          , 'Friday' : 'fri'
                                          , 'Saturday': 'sat'
                                          , 'Sunday': 'sun'}).fillna(0))

final_table = final_table[['group', 'PBID', 'mon', 'tue', 'wed', 'thu', 'fri'
                          , 'sat', 'sun', 'total-brushes', 'twice-brushes', 'vg-brush-time']]
final_table.to_csv('individual_metrics.csv')

##  Group Dynamics

In [375]:


# How many valid brush sessions were observed in total?

dataset2 = dataset.merge(groups, left_on = 'PlaybrushID', right_on = 'PBID')
total_sessions = pd.DataFrame(dataset2.groupby('group').sessions.count().rename('total_sessions'))

# What is the average number of brushing sessions per user in that group?

temp_count = pd.DataFrame(dataset2.groupby(['group', 'PBID']).sessions.count().rename('avg_sessions_per_user'))
user_sessions = temp_count.groupby('group').avg_sessions_per_user.mean()

# What is the average brushing duration per user in that group

temp_long = pd.DataFrame(dataset2.groupby(['group', 'PBID']).total_time.mean().rename('avg_time_per_user'))
user_long = temp_long.groupby('group').avg_time_per_user.mean()

In [376]:
group_data = (total_sessions.merge(user_sessions, on = 'group')
                            .merge(user_long, on = 'group')).reset_index()

In [383]:
group_data['performance'] =( (group_data.avg_time_per_user * group_data.avg_sessions_per_user)
                        / group_data.total_sessions)

group_data = group_data.sort_values('performance', ascending= False)

group_data['rank'] = group_data.performance.rank( axis = 0, ascending = False)

In [384]:
group_data

,group,total_sessions,avg_sessions_per_user,avg_time_per_user,performance,rank
2,C,157,6.826087,71.529584,3.109982,1.0
3,D,160,6.153846,71.857822,2.763762,2.0
1,B,61,2.772727,53.249442,2.420429,3.0
0,A,62,2.952381,45.367659,2.160365,4.0
